# 990PF 

In [4]:
import pandas as pd
from xml.etree import cElementTree as ElementTree
import xml.etree.ElementTree as et
from tqdm import tqdm
import logging

logging.basicConfig(format='%(asctime)s: %(message)s', filename='analysis.log', level=logging.DEBUG)

In [15]:
class XmlListConfig(list):
    def __init__(self, aList):
        for element in aList:
            if element:
                # treat like dict
                if len(element) == 1 or element[0].tag != element[1].tag:
                    self.append(XmlDictConfig(element))
                # treat like list
                elif element[0].tag == element[1].tag:
                    self.append(XmlListConfig(element))
            elif element.text:
                text = element.text.strip()
                if text:
                    self.append(text)


class XmlDictConfig(dict):
    '''
    Example usage:

    >>> tree = ElementTree.parse('your_file.xml')
    >>> root = tree.getroot()
    >>> xmldict = XmlDictConfig(root)

    Or, if you want to use an XML string:

    >>> root = ElementTree.XML(xml_string)
    >>> xmldict = XmlDictConfig(root)

    And then use xmldict for what it is... a dict.
    '''
    def __init__(self, parent_element):
        if parent_element.items():
            self.update(dict(parent_element.items()))
        for element in parent_element:
            if element:
                # treat like dict - we assume that if the first two tags
                # in a series are different, then they are all different.
                if len(element) == 1 or element[0].tag != element[1].tag:
                    aDict = XmlDictConfig(element)
                # treat like list - we assume that if the first two tags
                # in a series are the same, then the rest are the same.
                else:
                    # here, we put the list in dictionary; the key is the
                    # tag name the list elements all share in common, and
                    # the value is the list itself 
                    aDict = {element[0].tag: XmlListConfig(element)}
                # if the tag has attributes, add those to the dict
                if element.items():
                    aDict.update(dict(element.items()))
                self.update({element.tag: aDict})
            # this assumes that if you've got an attribute in a tag,
            # you won't be having any text. This may or may not be a 
            # good idea -- time will tell. It works for the way we are
            # currently doing XML configuration files...
            elif element.items():
                self.update({element.tag: dict(element.items())})
            # finally, if there are no child tags and no attributes, extract
            # the text
            else:
                self.update({element.tag: element.text})

def get_field(xmldict, path=None, default=None):
    if path is None: return xmldict
    try:
        for p in path:
            # Ugly namespaces are ugly, let's deal with them here
            xmldict = xmldict['{http://www.irs.gov/efile}' + p]
        return xmldict
    except KeyError:
        # If this key does not exist in the dictionary, 
        return default
        # Note: it may be faster to check ```if p not in dict``` rather
        # than set up this try/except block. I think it depends on how
        # often this fails - the more it fails, the worse the try/except does.

In [42]:
tree = et.parse("data/test/201722789349300202_public.xml")
root = tree.getroot()
#Converting from xml to dictionary
xmldict = XmlDictConfig(root)

test = get_field(xmldict,['ReturnData','IRS990ScheduleO','SupplementalInformationDetail'])
type(test)

__main__.XmlListConfig

In [20]:
def analyze_one(path):
    tree = et.parse(path)
    root = tree.getroot()
    #Converting from xml to dictionary
    xmldict = XmlDictConfig(root)

    
    ##############Header Information 
    #Date Information
    tax_period_begin = get_field(xmldict, ['ReturnHeader','TaxPeriodBeginDt'])
    
    #Filer Information
    filer_data = get_field(xmldict, ['ReturnHeader', 'Filer'], {})
    ein = get_field(filer_data, ['EIN'])
    business_name = get_field(filer_data, ['BusinessName', 'BusinessNameLine1Txt'])
    city = get_field(filer_data, ['USAddress', 'CityNm'])
    state = get_field(filer_data, ['USAddress', 'StateAbbreviationCd'])
    zipc = get_field(filer_data, ['USAddress', 'ZIPCd'])
    tax_year = get_field(xmldict, ['ReturnHeader', 'TaxYr'])
    filer_list = [[tax_period_begin, ein, business_name, city, state, zipc, tax_year]]
    filer_df = pd.DataFrame(filer_list, columns = ['TaxPeriodBeginDt','EIN', 'BusinessName', 'City', 'State', 'ZIPCd', 'TaxYr']) 
    
    
    ##############Return Data
    return_data = get_field(xmldict, ['ReturnData', 'IRS990PF'])
    if return_data is None:
        logging.warning(f'Could not find Return Data for file: {path}')
        return None
    
    #End of year assets "FMV of Assets at End of Year (required)"
    FMVAssetsEOYAmt = get_field(return_data, ['FMVAssetsEOYAmt'], "NA")
    
    #501c3 Exempt PF
    Organization501c3ExemptPFInd = get_field(return_data, ['Organization501c3ExemptPFInd'], "")
    
    #AdjustedNetIncomeAmt
    AdjustedNetIncomeAmt = get_field(return_data, ['AdjustedNetIncomeAmt'], "NA")
    
    
    
   


    return_list = [[FMVAssetsEOYAmt, Organization501c3ExemptPFInd, AdjustedNetIncomeAmt]]
    
    return_df = pd.DataFrame(return_list, columns = ['FMVAssetsEOYAmt', 'Organization501c3ExemptPFInd','AdjustedNetIncomeAmt'])

    #Concatenating header information and contributions
    return pd.concat([filer_df, return_df], axis=1, ignore_index=False)
    

In [21]:
#This is the method I used to get all the files in the directory

import os

'''
    For the given path, get the List of all files in the directory tree 
'''
def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles


dirName = 'data/test';
files = getListOfFiles(dirName)

In [22]:
df = pd.concat([one_row for name in tqdm(files) if (one_row := analyze_one(name)) is not None])
df


100%|██████████| 29/29 [00:00<00:00, 327.09it/s]


,TaxPeriodBeginDt,EIN,BusinessName,City,State,ZIPCd,TaxYr,FMVAssetsEOYAmt,Organization501c3ExemptPFInd,AdjustedNetIncomeAmt
0,2017-01-01,311815521,CARL AND HENRIETTA HERRMANN FAMILY,SOUTH PASADENA,CA,91030,2017,3647835,X,NA
0,2017-01-01,261924834,THE CHANG FAMILY FOUNDATION,ANN ARBOR,MI,48103,2017,0,X,NA
0,2016-07-01,161546071,THE RIGHTEOUS BABE FOUNDATION INC,NEW YORK,NY,10010,2016,187,X,NA
0,2017-01-01,300233845,CATHERINE MAYER FOUNDATION,SEATTLE,WA,981211358,2017,1261,X,NA
0,2017-01-01,043500902,JEFFRY AND SUSAN HOBBS-STEELE FAMILY,MILLIS,MA,02054,2017,2774,X,NA
0,2017-01-01,461362567,THE MIDDLEBROOKS-CAMP FOUNDATION,ATHENS,GA,30606,2017,248850,X,NA
0,2017-01-01,226781465,The Joan Perkowski Cashin Foundation,Jersey City,NJ,07304,2017,180152,X,NA
0,2017-01-01,262252920,EQUINE CHAMPIONS FOR CHRIST INC,HAMILTON,TX,76531,2017,21515,X,NA
